In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
path = f"D:/OCR/Automatic Number Plate Recognition/1. Introduction/1_Labeling"

In [3]:
path

'D:/OCR/Automatic Number Plate Recognition/1. Introduction/1_Labeling'

In [4]:
df = pd.read_csv( path + '/labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,D:/OCR/Automatic Number Plate Recognition/1. I...,1093,1396,645,727
1,D:/OCR/Automatic Number Plate Recognition/1. I...,134,301,312,350
2,D:/OCR/Automatic Number Plate Recognition/1. I...,31,139,128,161
3,D:/OCR/Automatic Number Plate Recognition/1. I...,164,316,216,243
4,D:/OCR/Automatic Number Plate Recognition/1. I...,813,1067,665,724


In [5]:
import xml.etree.ElementTree as xet

In [6]:
filename = df.filepath[0]
filename

'D:/OCR/Automatic Number Plate Recognition/1. Introduction/1_Labeling/images\\N1.xml'

In [7]:
def getFileName(filename):
    filename_img = xet.parse(filename).getroot().find('filename').text
    filepath_img = path +'/images/'+ filename_img
    return filepath_img

In [8]:
getFileName(filename)

'D:/OCR/Automatic Number Plate Recognition/1. Introduction/1_Labeling/images/N1.jpeg'

In [9]:
image_path = list(df.filepath.apply(getFileName))

In [10]:
file_path = image_path[5]
file_path

'D:/OCR/Automatic Number Plate Recognition/1. Introduction/1_Labeling/images/N104.jpeg'

In [11]:
img = cv2.imread(file_path)

cv2.rectangle(img,(1093, 645), (1396, 727), (0, 255, 0), 3)

cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

###  Data Preprocessing

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [14]:
labels = df.iloc[:2531,1:20].values

In [15]:
len(labels)

2531

In [16]:
image = image_path[0]
image

'D:/OCR/Automatic Number Plate Recognition/1. Introduction/1_Labeling/images/N1.jpeg'

In [17]:
len(image_path)//20

2531

In [18]:
data = []
output = []
for ind in range(len(image_path)//20):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h, w, d = img_arr.shape
    # Preprocessing
    load_image = load_img(image, target_size=(224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255 # Normalization
    # Normalization To Lebels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin, nxmax, nymin, nymax)
    ###--------------------Append--------------
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [19]:
x = np.array(data, dtype=np.float32)
y = np.array(output, dtype= np.float32)

In [20]:
x.shape, y.shape

((2531, 224, 224, 3), (2531, 4))

In [21]:
print('<-------------------Pickling----------------------->')
import pickle
# open a file where data need to be stored
file_x = open(path +'/list_x.pkl', 'wb')
file_y = open(path +'/list_y.pkl', 'wb')
# dump information to the file
pickle.dump(x, file_x)
pickle.dump(y, file_y)
# close the file
file_x.close()
file_y.close()

<-------------------Pickling----------------------->


In [22]:
print('<-------------------Un-Pickling----------------------->')
# open the file where data is dumped
file_x = open(path +'/list_x.pkl', 'rb')
file_y = open(path +'/list_y.pkl', 'rb')
# loading data
X = pickle.load(file_x)
Y = pickle.load(file_y)
# close the file
file_x.close()
file_y.close()

<-------------------Un-Pickling----------------------->


In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2024, 224, 224, 3), (507, 224, 224, 3), (2024, 4), (507, 4))

### Train Model Deep Learning

In [26]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

##### InceptionResNetV2 :
   is a convolutional neural network that is trained on more than a million images from the ___ImageNet database___.
   The network is 164 layers deep and can classify images into 1000 object categories.
##### ImageNet database :
   ImageNet is an image database organized according to the WordNet hierarchy
   
   
- include_top: 
        whether to include the fully-connected layer at the top of the network.
- weights: 
        one of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be   loaded.
- input_tensor: 
        optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.

In [39]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
model_v0 = inception_resnet.output
model_v0 = Flatten()(model_v0)
model_v0 = Dense(500,activation="relu")(model_v0)
model_v0 = Dense(250,activation="relu")(model_v0)
model_v0 = Dense(4,activation='sigmoid')(model_v0)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=model_v0)

In [40]:
#-------------Compile Model-----------------
model.compile(loss="mse", optimizer=Adam(learning_rate=1e-4))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 111, 111, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                                           

 batch_normalization_208 (Batch  (None, 25, 25, 96)  288         ['conv2d_208[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_210 (Batch  (None, 25, 25, 64)  192         ['conv2d_210[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_213 (Batch  (None, 25, 25, 96)  288         ['conv2d_213[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_214 (Batch  (None, 25, 25, 64)  192         ['conv2d_214[0][0]']             
 Normaliza

 activation_224 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_224[0][0]']
                                                                                                  
 conv2d_222 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_225 (Conv2D)            (None, 25, 25, 48)   13824       ['activation_224[0][0]']         
                                                                                                  
 batch_normalization_222 (Batch  (None, 25, 25, 32)  96          ['conv2d_222[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_225 (Batch  (None, 25, 25, 48)  144         ['conv2d_225[0][0]']             
 Normaliza

                                                                                                  
 activation_227 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_227[0][0]']
                                                                                                  
 activation_229 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_229[0][0]']
                                                                                                  
 activation_232 (Activation)    (None, 25, 25, 64)   0           ['batch_normalization_232[0][0]']
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_227[0][0]',         
                                                                  'activation_229[0][0]',         
                                                                  'activation_232[0][0]']         
          

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_243 (Batch  (None, 25, 25, 48)  144         ['conv2d_243[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_240 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_240[0][0]']
                                                                                                  
 activation_243 (Activation)    (None, 25, 25, 48)   0           ['batch_normalization_243[0][0]']
                                                                                                  
 conv2d_239 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_372 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_372[0][0]']
                                                                                                  
 conv2d_370 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_373 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_372[0][0]']         
                                                                                                  
 batch_normalization_370 (Batch  (None, 5, 5, 192)   576         ['conv2d_370[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_378 (Batch  (None, 5, 5, 192)   576         ['conv2d_378[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_381 (Batch  (None, 5, 5, 256)   768         ['conv2d_381[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_378 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_378[0][0]']
                                                                                                  
 activation_381 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_381[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_389 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_389[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_386[0][0]',         
                                                                  'activation_389[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 5, 5, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 5, 5, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_399 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_399 (Batch  (None, 5, 5, 192)   576         ['conv2d_399[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten_1 (Flatten)            (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense_3 (Dense)                (None, 500)          19200500    ['flatten_1[0][0]']              
                                                                                                  
 dense_4 (Dense)                (None, 250)          125250      ['dense_3[0][0]']                
                                                                                                  
 dense_5 (

In [29]:
y_train.shape

(2024, 4)

In [30]:
x_test.shape

(507, 224, 224, 3)

In [41]:
### Model Training 
from tensorflow.keras.callbacks import TensorBoard

In [42]:
tfb = TensorBoard('object_detection')

In [43]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100


KeyboardInterrupt: 

In [45]:
history = model.fit(x=x_train,y=y_train,batch_size=64,epochs=115,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/115
15/32 [=============>................] - ETA: 2:35 - loss: 0.0802

KeyboardInterrupt: 

In [ ]:
model.save(path +'/object_detection.h5')